In [226]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [227]:
data = pd.read_csv("data/train.csv")
data["is_test"] = 0
test = pd.read_csv("data/test.csv")
test["is_test"] = 1
data = data.append(test, sort=False)

In [228]:
data.sort_values(by=["year", "month", "day", "sched_dep_time"], inplace=True)
data["date"] = data.apply(lambda x: str(x["year"]) + "_" + str(x["month"]) + "_" + str(x["day"]), axis=1)

In [229]:
data["sched_arr_hr"] = data["sched_arr_time"].apply(lambda x: int(x / 100))

data["sched_dep_hr"] = data["sched_dep_time"].apply(lambda x: int(x / 100))

weather = pd.read_csv("data/weather.csv")
weather.drop_duplicates(subset=["year", "month", "day"], keep="first", inplace=True)
weather["date"] = weather.apply(lambda x: str(x["year"]) + "_" + str(x["month"]) + "_" + str(x["day"]), axis=1)
weather.drop(["year", "month", "day", "hour"], axis=1, inplace=True)

In [230]:
data = data.merge(weather, on=["origin", "date"], how="left")

In [231]:
data["precip"].fillna("NA", inplace=True)
data["precip"] = data["precip"].apply(lambda x: -999 if str(x) == "NA" else x)
data["temp"].fillna("NA", inplace=True)
data["temp"] = data["temp"].apply(lambda x: -999 if str(x) == "NA" else x)
data["humid"].fillna("NA", inplace=True)
data["humid"] = data["humid"].apply(lambda x: -999 if str(x) == "NA" else x)
data["wind_speed"].fillna("NA", inplace=True)
data["wind_speed"] = data["wind_speed"].apply(lambda x: -999 if str(x) == "NA" else x)
data["wind_gust"].fillna("NA", inplace=True)
data["wind_gust"] = data["wind_gust"].apply(lambda x: -999 if str(x) == "NA" else x)

In [232]:
len(date_vals)

336776

In [245]:
time_series1 = []
time_series2 = []
time_series3 = []
for i in range(len(data)):
    time_series1.append(i // 3)
    time_series1.append((i +1) // 3)
    time_series1.append((i +2) // 3)
data['time_feature1'] = time_series
data['time_feature2'] = time_series
data['time_feature3'] = time_series

In [247]:
delay_dict = {}
delay_dict_month = {}
delay_dict_tf1 = {}
delay_dict_tf2 = {}
delay_dict_tf3 = {}

origin_vals = data["origin"].values
date_vals = data["date"].values

month_vals = data["month"].values
tf_vals1 = data["time_feature1"].values
tf_vals2 = data["time_feature2"].values
tf_vals3 = data["time_feature3"].values

delay_vals = data["is_delayed"].values
is_test = data["is_test"].values
for n in range(0, len(data)):
    if origin_vals[n] not in delay_dict:
        delay_dict[origin_vals[n]] = {}
        delay_dict_month[origin_vals[n]] = {}
        delay_dict_tf1[origin_vals[n]] = {}
        delay_dict_tf2[origin_vals[n]] = {}
        delay_dict_tf3[origin_vals[n]] = {}
        
    if date_vals[n] not in delay_dict[origin_vals[n]]:
        delay_dict[origin_vals[n]][date_vals[n]] = {"total":0, "train":[]}
    
    if month_vals[n] not in delay_dict_month[origin_vals[n]]:
        delay_dict_month[origin_vals[n]][month_vals[n]] = {"total":0, "train":[]}
    
    if tf_vals1[n] not in delay_dict_tf1[origin_vals[n]]:
        delay_dict_tf1[origin_vals[n]][tf_vals1[n]] = {"total":0, "train":[]}
    
    if tf_vals2[n] not in delay_dict_tf2[origin_vals[n]]:
        delay_dict_tf2[origin_vals[n]][tf_vals2[n]] = {"total":0, "train":[]}
    
    if tf_vals3[n] not in delay_dict_tf3[origin_vals[n]]:
        delay_dict_tf3[origin_vals[n]][tf_vals3[n]] = {"total":0, "train":[]}
        
    if is_test[n] == 0:
        delay_dict[origin_vals[n]][date_vals[n]]["train"].append(delay_vals[n])
        
        delay_dict_month[origin_vals[n]][month_vals[n]]["train"].append(delay_vals[n])
        delay_dict_tf1[origin_vals[n]][tf_vals1[n]]["train"].append(delay_vals[n])
        delay_dict_tf2[origin_vals[n]][tf_vals2[n]]["train"].append(delay_vals[n])
        delay_dict_tf3[origin_vals[n]][tf_vals3[n]]["train"].append(delay_vals[n])
        
    delay_dict[origin_vals[n]][date_vals[n]]["total"] += 1
    
    delay_dict_month[origin_vals[n]][month_vals[n]]["total"] += 1
    delay_dict_tf1[origin_vals[n]][tf_vals1[n]]["total"] += 1
    delay_dict_tf2[origin_vals[n]][tf_vals2[n]]["total"] += 1
    delay_dict_tf3[origin_vals[n]][tf_vals3[n]]["total"] += 1

In [248]:
count_day = np.zeros(len(data))
delay_day = np.zeros(len(data))
delay_month = np.zeros(len(data))
delay_tf1 = np.zeros(len(data))
delay_tf2 = np.zeros(len(data))
delay_tf3 = np.zeros(len(data))

for n in range(0, len(data)):
    count_day[n] = delay_dict[origin_vals[n]][date_vals[n]]["total"]
    delay_day[n] = np.average(delay_dict[origin_vals[n]][date_vals[n]]["train"])
    delay_month[n] = np.average(delay_dict_month[origin_vals[n]][month_vals[n]]["train"])
    delay_tf1[n] = np.average(delay_dict_tf1[origin_vals[n]][delay_tf1[n]]["train"])
    delay_tf2[n] = np.average(delay_dict_tf2[origin_vals[n]][delay_tf2[n]]["train"])
    delay_tf3[n] = np.average(delay_dict_tf3[origin_vals[n]][delay_tf3[n]]["train"])
    
data["count_day"] = count_day
data["delay_day"] = delay_day
data["delay_month"] = delay_month
data["delay_tf1"] = delay_tf1
data["delay_tf2"] = delay_tf2
data["delay_tf3"] = delay_tf3

In [249]:
data["origin"] = pd.Categorical(data["origin"])
data["dest"] = pd.Categorical(data["dest"])
data["carrier"] = pd.Categorical(data["carrier"])

In [250]:
train = data[data["is_test"] == 0]
test = data[data["is_test"] == 1]

In [251]:
print(len(train), len(test))

168573 168203


In [252]:
use_features = ["distance", 
                "origin", 
                "dest", 
                "carrier", 
                "sched_arr_hr", 
                "sched_dep_hr", 
                "precip", 
                "temp", 
                "humid", 
                "wind_speed", 
                "count_day", 
                "delay_day",
                "delay_month",
                "delay_tf1",
                "delay_tf2",
                "delay_tf3",
                "visib",
                "time_feature"]

In [253]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 100,
    'max_depth': 10,
    'learning_rate': 0.05,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'verbose': -1,
}

In [254]:
train["is_val"] = np.random.randint(low=0, high=9, size=len(train))


/Users/falkvandermeirsch/Documents/DSR/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [255]:
lgb_train = lgb.Dataset(
    train[use_features].loc[train["is_val"]>1], 
    train["is_delayed"].loc[train["is_val"]>1])
lgb_eval = lgb.Dataset(
    train[use_features].loc[train["is_val"]<=1], 
    train["is_delayed"].loc[train["is_val"]<=1])
gbm = lgb.train(
    params, 
    lgb_train, 
    num_boost_round=5_000, 
    valid_sets=lgb_eval, early_stopping_rounds=100, verbose_eval=50)


Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.768598	valid_0's binary_logloss: 0.564852
[100]	valid_0's auc: 0.776163	valid_0's binary_logloss: 0.553756
[150]	valid_0's auc: 0.780456	valid_0's binary_logloss: 0.549076
[200]	valid_0's auc: 0.783426	valid_0's binary_logloss: 0.546106
[250]	valid_0's auc: 0.785147	valid_0's binary_logloss: 0.544312
[300]	valid_0's auc: 0.786205	valid_0's binary_logloss: 0.543244
[350]	valid_0's auc: 0.788137	valid_0's binary_logloss: 0.54135
[400]	valid_0's auc: 0.789061	valid_0's binary_logloss: 0.540448
[450]	valid_0's auc: 0.790091	valid_0's binary_logloss: 0.539428
[500]	valid_0's auc: 0.790914	valid_0's binary_logloss: 0.538608
[550]	valid_0's auc: 0.792061	valid_0's binary_logloss: 0.537472
[600]	valid_0's auc: 0.792749	valid_0's binary_logloss: 0.536788
[650]	valid_0's auc: 0.793261	valid_0's binary_logloss: 0.536206
[700]	valid_0's auc: 0.79372	valid_0's binary_logloss: 0.535784
[750]	valid_0's auc: 0.793956	

In [256]:
pred4 = gbm.predict(test[use_features])

In [259]:
test["is_delayed"] = 0.2*pred + 0.2*pred2 + 0.2*pred3 + 0.4*pred4
test[["id" ,"is_delayed"]].to_csv("sub14.csv", index=False)

/Users/falkvandermeirsch/Documents/DSR/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
